In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("probability_sampling.ipynb")

# What is in this file/notebook

Slides: https://docs.google.com/presentation/d/1vZC32UCamhyJWJBQIuP5AXKK896mBges_eO2H3Vo3q4/edit?usp=sharing

How do you use numpy's stats package to generalize a single random variable for:
a) Booleans (T/F)
b) Discrete variables (a, b, c)
c) Binned "continuous" variables - 0.1-0.2, 0.2-0.3, etc

Think of these as functions as simulating real-world events - query the sensor for if the door is open (y/n), ask where the robot is (contiuous location OR a grid square in the world), ask which room you're in (discrete variable, kitchen dining room, etc). These are all fancy versions of a coin toss (returns T/F with 50% probability each), a roll of the dice (returns 1..6 with equal probability).

ALL of these "simulate probability" routines can be implemented using JUST numpy's uniform number generator  (generates a number between 0 and 1 with all values equally likely). The simplest way to think of all of these methods is that you chop up the unit interval 0..1 into the number of possible outcomes, with each bit of the unit interval representing how likely that event is. Then you just generate a number from 0 to 1 and see which bin you fell into.

Why the functions are set up they way they are: You need to input how likely each discrete event is. There's three basic methods for specifying this.
1) List each discrete event and how likely it is
2) All events are equally likely, just say how many there are (bins) and the mapping between the bins and the 'labels'.Usually the bins represent some spatial variable like location or angle, but could be movement
3) There is a function that represents how likely each event is, with the x coordinate representing some continuous variable like distance (think Gaussian error for movement)

For each method that you'll implement the above information is passed in using a dictionary. I'm using a dictionary (instead of a class) because it's a bit easier to understand/implement, but the 'right' way to do this is as a class (see the last, optional, problem).

In [ ]:
# The imports you'll need
import numpy as np
import matplotlib.pyplot as plt


## Boolean
Simplest random variable - returns True or False, with some probability

Since probability of returning False is 1-prob(True), only need to specify one value.


In [ ]:
def sample_boolean_variable(info_variable):
    """ Generate one sample from a boolean variable
    @param info_variable is a dictionary containing the probability of the sensor returning True
    @returns True or False """

    # Probabilities have to be between 0 and 1...
    #. info_variable["prob_return_true"] accesses the value stored in the dictionary
    if info_variable["prob_return_true"] < 0.0 or info_variable["prob_return_true"] > 1.0:
        ValueError(f"Value {info_variable['prob_return_true']} not between zero and one")

    # First, use random.uniform to generate a number between 0 and one. Note that this is a uniform distribution, not
    #  a Gaussian one
    #.  random is a library in numpy (np), and uniform is a function in the random library
    #.  Google "numpy random"
    zero_to_one = np.random.uniform()

    # See slides - if the random variable is below the probability of returning true, return true. Otherwise, return false
    # TODO: Return True or False
    ...

In [ ]:
# First, check that you have no obvious syntax errors. This doesn't guarantee that your code is correct, just
#  that it doesn't crash and it returns True or False 
boolean_variable = {"prob_return_true": 0.7}
ret_val = sample_boolean_variable(boolean_variable)
if ret_val is True or ret_val is False:
    print("sample_boolean: Passed syntax check")


In [ ]:
# Test function: If your sample_boolean_variable is implemented correctly, then it should
#.  return "True" approximately test_prob_value percent of the time. This function directly
#.  checks that by, well, calling your function 10000 times and counting how many times it returned True...
def test_boolean(test_prob_value=0.6, n_samples=1000, b_print=True):
    """ Check if the sample_boolean is doing the right thing by calling it lots of times
    @param test_prob_value - any value between 0.0 and 1.0
    @param n_samples - how many samples to try. As this gets bigger, the percentage should get closer to test_prob_value
    @param b_print - whether or not to print out intermediate results
    @returns True if sample_boolean_variable is working correctly"""

    if b_print:
        print(f"Testing boolean with {test_prob_value} probability")
    boolean_info_variable = {"prob_return_true": test_prob_value}

    count_true = 0
    count_false = 0
    for _ in range(0, n_samples):
        if sample_boolean_variable(boolean_info_variable) == True:
            count_true += 1
        else:
            count_false += 1

    perc_true = count_true / (count_true + count_false)
    if b_print:
        print(f"Perc true from sampling: {perc_true}, expected {boolean_info_variable['prob_return_true']}")
    if not np.isclose(perc_true, boolean_info_variable["prob_return_true"], atol=0.05):
        if b_print:
            print("Failed")
        return False

    if b_print:
        print("Passed")
    return True

In [ ]:
# Note: This should return true and print out passed. However, sometimes the random number generator will not be
#  your friend and it will fail - you're expecting the count to come out around 0.6 +- noise
# This makes sure you are at least using the same seed to the pseudo random number generator
np.random.seed = 5
print(f"Boolean result: {test_boolean()}")

In [ ]:
grader.check("boolean_sample")

# Discrete

A list of discrete variables and their corresponding likelihood.

Because we need one value for each variable, and a name for each variable, store this as name/probabilty pair. Name is the key, probability is the value

If you have forgotten how to do dictionaries, go do the tutorial on dictionaries before attempting this problem

Hint: You want the .items() iterator. If you don't know what that is, or don't know what a key, value pair is, go do the tutorial.

In [ ]:
def sample_discrete_variable(info_variable):
    """ Generate one sample from the given discrete variable distribution
    Your code should NOT need to know what the actual keys are, how many there are, or what
    the actual values are - i.e., your code should NOT include things like
             if key == "True" or if z < 0.8
    @param info_variable contains pairs of values with probabilities. Probabilites should sum to one
    @returns one of the discrete values (keys) in the dictionary """

    # First, I'll do some checks for you
    for v in info_variable.values():
        # Probabilities have to be between 0 and 1...
        if v < 0.0 or v > 1.0:
            ValueError(f"Value {v} not between zero and one")

    # And they have to sum to one
    if not np.isclose(sum(info_variable.values()), 1.0):
        ValueError(f"Sum of probabilities should be 1, is {sum(info_variable.values())}")

    # Now, use random to generate a number between 0 and one
    zero_to_one = np.random.uniform()

    # See slides - "stack" the probabilities - if the value lies in the discrete value's stack, return that one
    # You should use a FOR loop. If you're struggling with the FOR loop, try writing this with an if - else if -else if
    #  for JUST the test case in the next cell - it won't work for test_discrete
    #  
    # TODO - return one of the key values in the dictionary. 
    ...

In [ ]:
# Syntax check - does your code run and return a key?
#.  Does NOT check if your code is correct, just that it returns one of the keys

check_discrete_tri = {"red": 0.2, "green": 0.5, "blue": 0.3}

#. Call 10 times
b_passed = True
for _ in range(0, 10):
    ret_value = sample_discrete_variable(check_discrete_tri)
    if not (ret_value == "red" or ret_value == "green" or ret_value == "blue"):
        b_passed = False
        print(f"Discrete: Failed syntax check, returned {ret_value}, expected red, green, or blue string")

if b_passed:
    print("Passed syntax check")

In [ ]:
def test_discrete(n_samples=10000, b_print=True):
    """ Test three different random variables
    @param n_samples - number of times to sample, bigger numbers are slower but more accurate
    @param b_print - set to True if you want a lot of stuff printed out
    @return True (passed) or False (did not pass)"""
    if b_print:
        print("Testing discrete")

    # The following for loop will loop through each of these in turn. It is NOT doing them all at the
    #  same time - the first time through the for loop it will check the boolean case, the second time
    #  the red, green, blue, the third time the quad one
    check_boolean = {"True": 0.6, "False": 0.4}
    check_discrete_tri = {"red": 0.2, "green": 0.5, "blue": 0.3}
    check_discrete_quad = {"kitchen": 0.2, "living room": 0.3, "dining room": 0.3, "bed room": 0.2}

    # check_variable will be check_boolean, then check_disrete_tri, then check_discrete_quad
    for check_variable in [check_boolean, check_discrete_tri, check_discrete_quad]:
        if b_print:
            print(f"Checking dictionary: {check_variable}")
        # For each discrete variable, set the counts to be zero; save as dictionary (rather than array/list) because
        #   the keys are strings
        counts = {}   # Empty dictionary
        for k in check_variable.keys():
            counts[k] = 0  # Add a key, 0 pair for each key

        # 'throw the dice' (sample) multiple times, and update counts as you go
        for _ in range(0, n_samples):
            # Get a sample from the distribution - should return one of the keys
            ret_key = sample_discrete_variable(check_variable)
            # Add one to that discrete variable's count
            counts[ret_key] += 1

        # Now compare the percentage values
        for k, v in check_variable.items():
            # How many times did I get this key?
            perc = counts[k] / n_samples
            if b_print:
                print(f"Discrete value: {k}, got: {perc}, expected {v}")

            if not np.isclose(perc, v, atol=0.05):
                if b_print:
                    print("Failed")
                return False
        if b_print:
            print(" Passed\n")
    return True

In [ ]:
# Note, this is a little slow
# If you want to speed it up, decrease number of samples
np.random.seed = 10
res = test_discrete(n_samples=10000, b_print=True)
if res:
    print(f"Passed discrete test")
else:
    print(f"Failed discrete test")

In [ ]:
grader.check("discrete")

# Binning

It is common to represent continuous variables as a set of discrete values (eg, on a scale of 1 to 10, how much do you like this problem?). This is common with locations (grid up the space), images (pixels), volume levels on most digital devices, etc. This is often called *binning*, because you are binning continuous values into discrete bins.

Creating a discrete random variable for a binned continuous variable is actually a special case of the discrete random variable - it's just that we don't need to explicitly label each discrete value. Instead the "labels" are the (typically) the value at the center of the bin. Rather than specifying unique labels for each bin, it's normal to just provide the start/stop boundaries and the number of divisions. The usual assumption is that all bins are equally likely; so if there are n bins, then the probability of generating any specific bin label is 1/n.

Your solution should NOT have a loop in it - you should be able to calculate which bin **zero_to_one** lies in directly (see np.floor(x)).

In [ ]:
# Note: this function should work even if value is NOT between start and stop
#  If value is start, should return 0
#  If value is stop, should return 1
#. If value is halfway between start and stop, should return 0.5
#.   and so on
def convert_start_stop_to_zero_one(value : float, start : float, stop : float):
    """ This takes in a number between start and stop and returns a number between 0 and 1 
    @param value : the value to convert
    @param start : left side of the interval
    @param stop : right side of the interval
    @preturn how far value is from start to stop"""

    # TODO return the percentage along
    ...

In [ ]:
# Checking your conversion
assert np.isclose(convert_start_stop_to_zero_one(3.0, 3.0, 6.0), 0.0)
assert np.isclose(convert_start_stop_to_zero_one(6.0, 3.0, 6.0), 1.0)
assert np.isclose(convert_start_stop_to_zero_one(4.5, 3.0, 6.0), 0.5)
assert np.isclose(convert_start_stop_to_zero_one(0.0, -2.0, 2.0), 0.5)

In [ ]:
# Note: this function should work even if value is NOT between 0 and 1
#  If value is 0, should return start
#  If value is 1, should return stop
#. If value is 0.5, should retrub halfway between start and stop
#.   and so on
def convert_zero_one_to_start_stop(value : float, start : float, stop : float):
    """ This takes in a number between 0 and 1 and returns a number between start and stop 
    @param value : the value to convert
    @param start : left side of the interval
    @param stop : right side of the interval
    @preturn how far value is from 0 to 1"""

    # TODO return the percentage between start and stop
    ...

In [ ]:
assert np.isclose(convert_zero_one_to_start_stop(0.0, 3.0, 6.0), 3.0)
assert np.isclose(convert_zero_one_to_start_stop(1.0, 3.0, 6.0), 6.0)
assert np.isclose(convert_zero_one_to_start_stop(0.5, 3.0, 6.0), 4.5)
assert np.isclose(convert_zero_one_to_start_stop(0.5, -2.0, 2.0), 0.0)

In [ ]:
# 
def sample_bin_variable(info_variable):
    """Sample from the discrete bin variable. The "label" to return is the center location of the bin 
    Assumes all bins are equally likely (which is why there is not specific probability value given)
    @param info_variable - bin start and stop, number of bins. Keys: start, stop, n_bins
    @return The value (center) associated with the bin"""

    zero_to_one = np.random.uniform()
    # TODO:
    #  Step 1: Calculate the size of each bin ON THE UNIT INTERVAL
    #  Step 2: Use np.floor to find the INDEX of the bin - do NOT use a for loop
    #  Step 3: Calculate the center of the bin with that index on the (start, stop) interval
    #.   Reminder that if you have 1 bin on the interval 0 to 1, then you would return 0.5 (the center of the bin)
    ...

In [ ]:
# Checking the syntax of the call
check_bins = {"start": -2.0, "stop": 3.0, "n_bins": 10}
bin_loc = sample_bin_variable(check_bins)
if check_bins["start"] < bin_loc < check_bins["stop"]:
    print("bin sampling: return value is in correct range")

In [ ]:
# Check returns center of bin 0 or bin 1
check_return_center = {"start": 0.0, "stop": 1.0, "n_bins": 2}
bin_loc = sample_bin_variable(check_return_center)
if not (np.isclose(bin_loc, 0.25) or np.isclose(bin_loc, 0.75)):
    print(f"If two bins, the first bin goes from 0 to 0.5, the second from 0.5 to 1.0, so return either 0.25 or 0.75 (yours was {bin_loc})")

In [ ]:
def test_bins(n_samples=10000, b_print=True):
    if b_print:    
        print("Testing bins")
    # Provide the start and stop values, and the number of bins
    check_bins = {"start": -2.0, "stop": 3.0, "n_bins": 10}

    counts = np.zeros(check_bins["n_bins"])

    bin_width = (check_bins["stop"] - check_bins["start"]) / check_bins["n_bins"]
    for _ in range(0, n_samples):
        # Which bin location was selected?
        bin_loc = sample_bin_variable(check_bins)

        # Convert back to the bin id
        bin = int(np.floor((bin_loc - check_bins["start"]) / bin_width))
        
        # Add one to that bin count
        counts[bin] += 1

    # All of the percentage values should be the same
    perc_expected = 1.0 / check_bins["n_bins"]
    for i, count in enumerate(counts):
        perc_found = count / n_samples
        bin_loc = check_bins["start"] + (i + 0.5) * bin_width
        if b_print:
            print(f"Bin loc {bin_loc} perc {perc_found} expected close to {perc_expected}")

        if not np.isclose(perc_found, perc_expected, atol=0.05):
            if b_print:
                print("Failed")
            return False
    if b_print:
        print("Passed")
    return True

In [ ]:
print(f"Bin result: {test_bins()}")

In [ ]:
grader.check("bins")

## Gaussian sampling
This is a generic Gaussian noise variable - I'm including it here because you'll need it in subsequent assignments. But it's basically "store mu and sigma, then use those to generate noise"

In [ ]:
def sample_gaussian_variable(info_variable):
    """Return a sample from the Gaussian
    @param info_variable - mu and sigma
    @return A sample from the Gaussian"""

    # Call random.normal here
    # TODO: Call np.random.normal here and return the number
    ...

In [ ]:
# Checking syntax of call
check_gaussian = {"mu": 1.2, "sigma": 0.2}
sample = sample_gaussian_variable(check_gaussian)
print(f"Sample value should be a number: {sample}")

In [ ]:
def test_gaussian(n_samples=50000, b_print=True):
    """Test the gaussian distribution by seeing if the mean/sd are the same
    @param b_print print out test results y/n
    """
    if b_print:
        print("Testing Gaussian")
    # Provide mu and sigma
    check_gaussian = {"mu": 1.2, "sigma": 0.2}

    # This does the for loop "in one line" - read this as
    #   for _ in range()
    #       sample_gaussian...
    samples = [sample_gaussian_variable(check_gaussian) for _ in range(0, n_samples)]

    # Should get out same mu/sigma
    samples_mean = np.mean(samples)
    samples_sigma = np.std(samples)

    if not np.isclose(samples_mean, check_gaussian["mu"], atol=0.05):
        raise ValueError(f"Failed Gaussian, expected {check_gaussian['mu']}, got {samples_mean}")

    if not np.isclose(samples_sigma, check_gaussian['sigma'], atol=0.05):
        raise ValueError(f"Failed Gaussian, expected {check_gaussian['sigma']}, got {samples_sigma}\n")

    if b_print:
        print("Passed\n")
    return True

In [ ]:
print(f"Gaussian result: {test_gaussian()}")

In [ ]:
# Consider a time of flight distance sensor that you are "testing" with two distances; 2cm and 20cm. 
# The standard deviation of the noise at 2cm is 1mm, and at 20cm it is 2cm. At a distance of 20cm the sensor is also
#.  biased by 3.0cm - i.e., on average, if something is 20cm away it will say it is 23.0cm away

# Scipy stats has a function that takes in a mean and a standard deviation and returns a function that is that Gaussian distribution
from scipy.stats import norm


#  TODO Create plots (using np.random.normal()) that show what the expected error for the sensor at 2cm and 20cm
#.    Don't forget labels
# Note: Does it make sense to plot for distances < 0? 
fig, axs = plt.subplots(1, 2)

...

# Example of plotting a normal function - edit this
# How to create 
rv = norm(loc=0.0, scale=1.0)

ts = np.linspace(-1.0, 1.0)
axs[0].plot(ts, rv.pdf(ts))
fig.tight_layout()

In [ ]:
grader.check("Gaussian")

# Probability mass function (discrete) 

This is a more general version of the previous bin variable, with the main difference being that each bin has a different liklihood (as specified by the input function). So it's a combination of the discrete variable (using a running sum to determine which bin you fall in) and the bins (chopping up a continuous variable into bins).

Technical note: In theory land, there is a difference between doing this as a continuous function (probability density) versus chopping it up into pieces (probability mass). You can actually do continuous functions, but it's a bit trickier and we don't need it (see for example https://www.comsol.com/blogs/sampling-random-numbers-from-probability-distribution-functions/)

For this example we're going to use a class instead of a method because (in order to make it efficient) you want to pre-calculate a running sum from the given probabilities. It would be very expensive to do this every time you asked for a sample, like you did in the discrete problem.

This is also a good time to do some fancy numpy array stuff, namely, using "where" to find the index (instead of writing your own for loop)

In [ ]:
class SampleProbabilityMassFunction:
    def __init__(self, in_pdf, x_range=(0.0, 1.0), n_bins=100):
        """ Given a probability mass function, what range of x to use, and the number of samples, create the running
        sum/data needed to generate random samples from that pmf
        @param in_pdf - the function representing the probability distribution
        @param x_range - min and max x values as a tuple
        @param n_bins - number of bins """

        # To "keep" a variable, use self.variable_name
        self.start = x_range[0]
        self.end = x_range[1]
        self.n_bins = n_bins

        # TODO - Initialize correctly
        #  Where the bins start and end
        #.     Where the center of each bin is
        #  The amount of probability to put in each bin
        #.     Sample the in_pdf function at the center of the bin
        #.     Normalize the probabilities so they sum to 1
        #  The running sum
        #.     Calculate the left (and right) boundaries on the unit interval for each bin

        # This is an example of making a numpy array of size n_bins (filled with zeros)
        self.bin_centers = np.zeros(shape=(n_bins))
        
        # Create the pmf by evaluating in_pdf at the center of each bin
        #   Don't forget to normalize - the sum of self.bin_heights should be 1

        # Running sum of probabilities - bin_sum[i] = sum(bin_heights[0:i])
        #  Note: It's a bit easier to generate_sample if you make this array n_bins+1, with the first value being 0
        #   and the last value being 1         

    def get_bin_center(self, indx : int):
        """ Return the center of bin indx (i.e., in our 2 bin on the unit interval example, 
        0 would return 0.25 and 1 would return 0.75)
        @param indx : which bin
        @return bin center"""
        # TODO Return the bin center
        ...

    def get_bin_probability(self, indx : int):
        """ Return the probability of that bin being selected
        @param indx : which bin
        @return probability"""
        # TODO return the probability
        ...
    
    def generate_sample(self):
        """ Draw one sample from the pmf
        Very similar to the discrete example above, for picking which bin, except you've pre-calculated the running sum.
        Very similar to bin_sample for returning the bin center, exept you've pre-calculated the bin centers
        @return bin center """
        zero_to_one = np.random.uniform()

        # You want the index i where bin_sum[i] <= zero_to_one < bin_sum[i+1]
        # Not fancy version: Use a for loop
        # Fancy version: Use np.where - see tutorial on where
        # TODO - return correct bin center               
        ...

    def _generate_counts(self, n_samples):
        """ Generate n samples
        @param n_samples - number of samples per bin
        @returns a numpy array with the counts for each bin, normalized"""

        # Counts
        counts = np.zeros(self.n_bins)


        #. call generate_sample many times
        for _ in range(0, self.n_bins * n_samples):
            x_value = self.generate_sample()
            # Should be between 0 and n_bins - 1
            assert x_value > self.start and x_value < self.end
            # Convert x value to 0, 1
            x_value_zero_one = convert_start_stop_to_zero_one(x_value, self.start, self.end)
            # convert 0, 1 value to n_bins
            x_value_bin_indx = x_value_zero_one * self.n_bins
            bin_index = np.floor(x_value_bin_indx)
            counts[int(bin_index)] += 1.0

        # Normalize
        counts = counts / sum(counts)
        return counts

    def test_self(self, in_pdf, b_print=True):
        """ Check/test function
        @param in_pdf - the pdf function used to generate the values
        @returns True/False"""

        # Expected probability values
        bin_centers = np.zeros((self.n_bins))
        for indx in range(0, self.n_bins):
            bin_centers[indx]= self.get_bin_center(indx)

        # Evaluate the function in_pdf at the bin centers 
        expected_probs = in_pdf(self.bin_centers)
        
        # Normalize
        expected_probs /= np.sum(expected_probs)

        # Call the generate_sample method many, many times
        counts = self._generate_counts(n_samples=100)

        # This cute trick loops over expected probabilities and counts at the same time
        #.   requires that the two be arrays of the same size
        for exp, c in zip(expected_probs, counts):
            if b_print:
                print(f"pmf perc {c} expected {exp}")

            if np.abs(exp - c) > 0.1:
                print("Failed")
                return False

        if b_print:
            print("Passed")
        return True


In [ ]:
def pdf(x):
    """ Made-up pdf (a quadratic). Can be anything, as long as it's not negative
    @param x
    @ return (x+1) * (x+1) + 0.1"""
    return (x+1) ** 2 + 0.1

In [ ]:
# Syntax check
x_min = -2.0
x_max = 1.0
n_bins = 10

# Make the class. Reminder, if you change the class, you need to re-execute the above cell
my_sample = SampleProbabilityMassFunction(pdf, (x_min, x_max), n_bins)
# Generate a sample
ret_value = my_sample.generate_sample()
if x_min < ret_value < x_max:
    print("PMF: Passed syntax check")

In [ ]:
# TODO: Write some more test code to see if your class is doing the right thing...

In [ ]:
# Now check that it actually returns the correct counts
my_pmf = SampleProbabilityMassFunction(in_pdf=pdf, x_range=(-2.0, 1.0), n_bins=10)

# Call the test function on the class
assert my_pmf.test_self(in_pdf=pdf)

In [ ]:
# This is just to help you visualize what is going on/what you're trying to do
fig_2, axs_2 = plt.subplots(1, 3)

# First, plot the original, continuous pdf NORMALIZED
ts_pdf = np.linspace(my_sample.start, my_sample.end, 4 * my_sample.n_bins)
ys_pdf = pdf(ts_pdf)
axs_2[0].plot(ts_pdf, ys_pdf, '-k', label="pdf")
axs_2[0].set_title("Original PDF\n(no normalization)")
axs_2[0].legend()

# Now do the normalized plot and your pmf
ys_pdf_normalized = ys_pdf / np.sum(ys_pdf)
axs_2[1].plot(ts_pdf, ys_pdf_normalized , '-k', label="normalized pdf")

check_bin_centers = np.zeros((my_sample.n_bins))
check_bin_values = np.zeros((my_sample.n_bins))
for indx in range(0, my_sample.n_bins):
    check_bin_centers[indx] = my_sample.get_bin_center(indx)
    check_bin_values[indx] = my_sample.get_bin_probability(indx)
axs_2[1].plot(check_bin_centers, check_bin_values, 'gX', label="PMF")
axs_2[1].set_title("Normalized PDF\n and discretized PDF")
axs_2[1].legend()

# Reconstructed counts
counts = my_sample._generate_counts(n_samples=100)
axs_2[2].set_title("pdf to pmf")

axs_2[2].plot(check_bin_centers, check_bin_values , '-k', label="PMF")
axs_2[2].plot(check_bin_centers, counts, 'ob', label="Sampled PMF")
axs_2[2].legend()

print(f"Should both be 1.0: {np.sum(ys_pdf_normalized)}, {np.sum(check_bin_values)}")
fig_2.tight_layout()

In [ ]:
grader.check("pmf")

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# TODO - set to correct value               
# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS FW25(creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5


In [ ]:
grader.check("hours_collaborators")

### To submit

- Do a restart then run all to make sure everything runs ok
- Save the file
- Submit this .ipynb file through gradescope, Homework 1
- Take out/suppress all print statements

If the Gradescope autograder fails, please check here first for common reasons for it to fail
    https://docs.google.com/presentation/d/1tYa5oycUiG4YhXUq5vHvPOpWJ4k_xUPp2rUNIL7Q9RI/edit?usp=sharing

